In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
dir_path = Path.cwd().parent
data_folder = dir_path.joinpath("data")
models_folder = dir_path.joinpath("data/models/2024")

# west_low_species = [12, 11, 8, 7, 6, 5, 4, 3, 1] #target
# mid_line_species = [13, 10, 9, 2, 0] #control

west_low_species = [8] #target
mid_line_species = [12, 10] #control

model_df = data_folder.joinpath("processed/model_df_2024.parquet")

print(model_df)

/home/bippw1/bipp/ncount/sentinel-2-grab/data/processed/model_df_2024.parquet


In [12]:
# def rhodo_balance(data_frame):
#     """Ensuring pine is limited to 3x of Rhododendron count"""
#     #rhododendron = 28,590

#     rhodo_df = data_frame[data_frame['species']==0]
#     pine_df = data_frame[data_frame['species']==1]

#     set_seed = 4526
#     pine_df_holder = []
#     for pine_id, pine_df in pine_df.groupby('year'):
#         sampled_pine_df = []

#         sampled_pine_df = pine_df.sample(n=14000, random_state=set_seed).reset_index(drop=True)

#         pine_df_holder.append(sampled_pine_df)

#     pine_model_df = pd.concat(pine_df_holder, axis=0)
#     final_df = pd.concat([pine_model_df, rhodo_df])
#     return final_df

In [3]:
def get_species(target: int, control: int, out_folder: Path):
    out_file_name = f"{target}_{control}.parquet"
    out_path = out_folder.joinpath(out_file_name)

    print(f"{target}_{control}: Createing Model Frame")
    df = pd.read_parquet(model_df)

    df_species_filtered = df[df["species"].isin([target, control])]

    df_species_filtered["species"] = np.where(
        df_species_filtered["species"] == target, 1, 0
    )

    print(df_species_filtered['species'].value_counts())

    # if control == 10:
    #     df_species_filtered = rhodo_balance(df_species_filtered)
    
    df_species_filtered.to_parquet(out_path, index=False)

    # print(df_species_filtered)
        

In [4]:


for target in west_low_species:
    for control in mid_line_species:
        # Creating a species based folder to hold the model and related outputs
        species_folder = models_folder.joinpath(f"{target}_{control}")
        if not species_folder.exists():
            species_folder.mkdir(parents=True)

        get_species(target=target, control=control, out_folder=species_folder)

8_12: Createing Model Frame
species
1    5000
0    5000
Name: count, dtype: int64
8_10: Createing Model Frame
species
1    5000
0    4765
Name: count, dtype: int64


/tmp/ipykernel_14494/3497029953.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_species_filtered["species"] = np.where(
